# WOfS Validation Count <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary)

## Background
Accuracy assessment for WOfS product in Africa includes generating a confusion error matrix for a WOFL binary classification.
The inputs for the estimating the accuracy of WOfS derived product are a binary classification WOFL layer showing water/non-water and a shapefile containing validation points collected by [Collect Earth Online](https://collect.earth/) tool. Validation points are the ground truth or actual data while the extracted value for each location from WOFL is the predicted value. A confusion error matrix containing overall, producer's and user's accuracy is the output of this analysis. 

## Description
This notebook explains how you can perform accuracy assessment for WOFS derived product using collected ground truth dataset. 

The notebook demonstrates how to:
1. Generating a confusion error matrix for WOFL binary classification
2. Assessing the accuracy of the classification 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages
Import Python packages that are used for the analysis.

In [10]:
%matplotlib inline

import time 
import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from rasterio.mask import mask
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import plot_confusion_matrix, f1_score  
from deafrica_plotting import map_shapefile,display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_dask import create_local_dask_cluster

### Loading Dataset

Read in the validation data csv, clean the table and rename the column associated with actual and predicted. 

We need to read two columns from this table:
- Water flag as the groundtruth(actual)
- Class Wet from WOfS (prediction)

### Joining CEO tables and Counting the number of Points 

In [155]:
#joining dataframes together and extract one csv for each partner institution 
DF = glob.glob('../Supplementary_data/Validation/Refined/NewAnalysis/CEO_*.csv')
frame = []
for d in DF: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
Africa = pd.concat(frame)

In [156]:
Africa.to_csv(('../Supplementary_data/Validation/Refined/NewAnalysis/Africa_ValidationPoints.csv'))

In [157]:
AfricaCount = Africa.groupby('PLOT_ID',as_index=False,sort=False).last()
AfricaCount

,PLOT_ID,Unnamed: 0,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG
0,137483175.0,0,30.463813,-26.653807,0.0,1.0,2018,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,2,2
1,137483176.0,1,30.026031,-26.673227,0.0,1.0,2018,NaN,NaN,1-12,0,0,0,Open water - Constructed (e.g. aquaculture),NaN,12,1
2,137483177.0,2,31.700362,-26.746737,0.0,1.0,2018,NaN,NaN,1-12,0,"1,2",0,Open water - freshwater,NaN,2,2
3,137483178.0,3,31.937287,-26.801901,0.0,1.0,2018,NaN,NaN,1-12,0,"2,7,12",0,Open water - freshwater,NaN,12,2
4,137483179.0,4,27.339949,-26.863925,0.0,1.0,2018,NaN,NaN,1-12,0,0,0,Open water - freshwater,NaN,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,137387432.0,195,39.613404,-9.090942,0.0,1.0,2018,NaN,NaN,"1-4,6-12",0,0,5,Open water - marine,NaN,5,3
2696,137387433.0,196,35.464420,-9.505899,0.0,1.0,2018,NaN,NaN,"1,2,4-6,8,10,12",0,"3,9,11",7,Open water - freshwater,NaN,7,3
2697,137387434.0,197,33.947728,-9.568113,0.0,1.0,2018,NaN,NaN,2-12,0,1,0,Open water - freshwater,NaN,1,2
2698,137387435.0,198,34.424376,-9.843084,0.0,1.0,2018,NaN,NaN,0,"1,2,7,8-11","3,4,5,6,12",0,Forest/woodlands,NaN,12,2


### Joining WOfS-based Analysed Tables and Counting the number of Points 

In [63]:
#joining dataframes together and extract one csv for each partner institution 
DF2 = glob.glob('../Supplementary_data/Validation/Refined/Continent/AEZ_count/Groundtruth_*.csv')
frame = []
for d in DF2: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
Analysed = pd.concat(frame)

In [64]:
Analysed.to_csv(('../Supplementary_data/Validation/Refined/Continent/AEZ_count/Africa_AnalysedPoints.csv'))

In [61]:
AnalysedCount = Analysed.groupby('PLOT_ID',as_index=False,sort=False).last()
# #AnalysedCount

### Joining Inland Tables and Counting the number of Points 

In [65]:
#joining dataframes together and extract one csv for each partner institution 
DF3 = glob.glob('../Supplementary_data/Validation/Refined/Continent/AEZ_count/inland_*.csv')
frame = []
for d in DF3: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
Inland = pd.concat(frame)

In [66]:
Inland.to_csv(('../Supplementary_data/Validation/Refined/Continent/AEZ_count/Africa_InlandPoints.csv'))

In [67]:
InlandCount = Inland.groupby('PLOT_ID',as_index=False,sort=False).last()
InlandCount

,PLOT_ID,Unnamed: 0,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION,geometry
0,137755802.0,1,-0.260555,16.620471,I am unsure,5,1,1.0,2.0,1,POINT (-0.260555179 16.62047074)
1,137755803.0,3,-14.230417,16.545471,I am unsure,3,1,1.0,2.0,1,POINT (-14.23041697 16.54547113)
2,137755804.0,4,-15.681815,16.479809,Wetlands - freshwater,3,1,1.0,2.0,1,POINT (-15.68181503 16.47980856)
3,137755805.0,5,-0.115353,16.473684,Wetlands - freshwater,3,1,1.0,2.0,1,POINT (-0.115353188 16.47368443)
4,137755806.0,8,-3.217452,16.388211,Wetlands - freshwater,11,1,1.0,1.0,1,POINT (-3.217452255 16.38821123)
...,...,...,...,...,...,...,...,...,...,...,...
2180,137387430.0,6068,37.568139,-9.073325,I am unsure,8,1,0.0,1.0,0,POINT (37.56813912 -9.073325318)
2181,137387433.0,6083,35.464420,-9.505899,Open water - freshwater,12,1,1.0,4.0,1,POINT (35.46442034 -9.505899182)
2182,137387434.0,6093,33.947728,-9.568113,Open water - freshwater,12,1,1.0,2.0,1,POINT (33.94772802 -9.568112982999999)
2183,137387435.0,6095,34.424376,-9.843084,Forest/woodlands,9,0,0.0,1.0,0,POINT (34.4243761 -9.843083996000001)


### Reading Continental Validation points and Extract Count for the desired AEZ 

In [69]:
ValPoints = '../Supplementary_data/Validation/Refined/NewAnalysis/Groundtruth/Groundtruth_Africa.csv'
df = pd.read_csv(ValPoints,delimiter=",")

In [70]:
geometry = [Point(xy) for xy in zip(df.LON, df.LAT)]
crs = {'init': 'epsg:4326'} 
AfricaValPoints = GeoDataFrame(df, crs=crs, geometry=geometry)

In [71]:
#Defining the agro-echological zone
aez = '../Supplementary_data/Validation/Refined/Continent/shapefile/AEZs_simple_Northern.shp'

In [72]:
outline = gpd.read_file(aez).to_crs('EPSG:4326')
outline

,Zone,AreaSQKM,geometry
0,Northern,5.746049e+06,"POLYGON ((9.75594 37.34962, 9.76010 37.33980, ..."


In [73]:
#Clipping the input data with the buffer zone to have all inland points that are not coastal 
Zone_points = gpd.clip(AfricaValPoints, outline)

In [74]:
Zone_points.to_csv(('../Supplementary_data/Validation/Refined/NewAnalysis/Counts/ValidationPoints_Northern.csv'))

In [75]:
Zone_points

,Unnamed: 0,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2Y,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,CLASS_WET,CLEAR_OBS,geometry
2859,2859,137755561.0,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,1,0,0.0,2.0,POINT (-8.84768 30.99664)
2860,2860,137755561.0,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,2,0,1.0,2.0,POINT (-8.84768 30.99664)
2861,2861,137755561.0,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,4,0,0.0,1.0,POINT (-8.84768 30.99664)
2862,2862,137755561.0,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,5,0,0.0,0.0,POINT (-8.84768 30.99664)
2863,2863,137755561.0,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,6,0,0.0,1.0,POINT (-8.84768 30.99664)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8634,8634,137755560.0,32.430103,31.028901,0.0,1.0,2018,1/07/2018,5/07/2018,0,"2, 4, 6, 7","1, 3, 5, 8, 9, 10, 11, 12",0,Cultivated (Cropland/ Plantation),NaN,8,2,1.0,2.0,POINT (32.43010 31.02890)
8635,8635,137755560.0,32.430103,31.028901,0.0,1.0,2018,1/07/2018,5/07/2018,0,"2, 4, 6, 7","1, 3, 5, 8, 9, 10, 11, 12",0,Cultivated (Cropland/ Plantation),NaN,9,2,0.0,0.0,POINT (32.43010 31.02890)
8636,8636,137755560.0,32.430103,31.028901,0.0,1.0,2018,1/07/2018,5/07/2018,0,"2, 4, 6, 7","1, 3, 5, 8, 9, 10, 11, 12",0,Cultivated (Cropland/ Plantation),NaN,10,2,0.0,0.0,POINT (32.43010 31.02890)
8637,8637,137755560.0,32.430103,31.028901,0.0,1.0,2018,1/07/2018,5/07/2018,0,"2, 4, 6, 7","1, 3, 5, 8, 9, 10, 11, 12",0,Cultivated (Cropland/ Plantation),NaN,11,2,0.0,0.0,POINT (32.43010 31.02890)


In [76]:
ZoneCount_org = Zone_points.groupby('PLOT_ID',as_index=False,sort=False).last()
ZoneCount_org

,PLOT_ID,Unnamed: 0,LON,LAT,FLAGGED,ANALYSES,SENTINEL2Y,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,CLASS_WET,CLEAR_OBS,geometry
0,137755561.0,2870,-8.847683,30.996639,0.0,1.0,2018,2018-02-01,2018-02-05,0,"1, 2, 4, 5, 6, 7, 9, 11, 12","3, 8, 10",0,Barren (Bare soil / Rocky Land),NaN,10,2,1.0,2.0,POINT (-8.84768 30.99664)
1,137755562.0,2882,29.639551,30.982425,0.0,1.0,2018,2018-06-01,2018-06-05,"1,2, 3, 4, 7, 9, 10, 11",0,"5, 8, 12",6,I am unsure,NaN,6,3,0.0,0.0,POINT (29.63955 30.98242)
2,137755563.0,2894,31.242680,30.954004,0.0,1.0,2018,2018-12-01,2018-12-05,"1,2,3,4","9,10,11",5,"6,7,8,12",Wetlands - freshwater,NaN,12,3,1.0,2.0,POINT (31.24268 30.95400)
3,137755564.0,2906,-7.271915,30.949085,0.0,1.0,2018,2018-01-01,2018-01-05,"3,11,12","7,9","4,8","1,2,5,6,10",Wetlands - freshwater,NaN,10,3,0.0,1.0,POINT (-7.27191 30.94909)
4,137755565.0,2918,-6.064282,30.936518,0.0,1.0,2018,2018-12-01,2018-12-05,0,"1,4,5,6,7,8,9,10","2,3,11,12",0,Barren (Bare soil / Rocky Land),NaN,12,2,0.0,2.0,POINT (-6.06428 30.93652)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,137755556.0,8590,33.514920,31.119182,0.0,1.0,2018,1/09/2018,5/09/2018,0,0,"1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12",3,I am unsure,NaN,3,3,0.0,0.0,POINT (33.51492 31.11918)
296,137755557.0,8602,32.922608,31.067739,0.0,1.0,2018,1/08/2018,5/08/2018,0,0,"3, 5, 6, 7, 8, 10, 11, 12","1, 2, 4, 9",I am unsure,NaN,9,3,1.0,1.0,POINT (32.92261 31.06774)
297,137755558.0,8614,30.981503,31.046950,0.0,1.0,2018,1/07/2018,5/07/2018,"2, 3, 4, 6, 7, 8, 9, 10, 11, 12",0,"1, 5",0,Open water - freshwater,NaN,5,2,0.0,0.0,POINT (30.98150 31.04695)
298,137755559.0,8626,29.759257,31.046130,0.0,1.0,2018,1/06/2018,5/06/2018,"1, 2, 3, 4, 5, 6, 7, 9, 10, 11",0,"8, 12",0,Open water - Constructed (e.g. aquaculture),NaN,12,2,1.0,2.0,POINT (29.75926 31.04613)


### Institution-based Count 

In [153]:
#Read the ground truth data following cleaning step 
#CEO = '../Supplementary_data/Validation/Refined/AFRIGIST/CEO_AFRIGIST_2020-09-15.csv'
CEO = '../Supplementary_data/Validation/Refined/NewAnalysis/CEO_2_RCMRD_2020-07-30.csv'
df1= pd.read_csv(CEO,delimiter=",")

In [154]:
CEOCount= df1.groupby('PLOT_ID',as_index=False,sort=False).last()
CEOCount

,PLOT_ID,Unnamed: 0,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG
0,137387237.0,0,36.248262,-0.439987,0.0,1.0,2018,"1-3,5-12",0,4,0,Wetlands - freshwater,Possible high algae bloom between 1-3,4,2
1,137387238.0,1,34.149518,-0.539462,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,NaN,12,1
2,137387239.0,2,34.159779,-0.542990,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,NaN,12,1
3,137387240.0,3,30.925848,-0.669983,0.0,1.0,2018,1-11,0,12,0,Open water - freshwater,NaN,12,2
4,137387241.0,4,37.892745,-0.683152,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,NaN,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,137387432.0,195,39.613404,-9.090942,0.0,1.0,2018,"1-4,6-12",0,0,5,Open water - marine,NaN,5,3
196,137387433.0,196,35.464420,-9.505899,0.0,1.0,2018,"1,2,4-6,8,10,12",0,"3,9,11",7,Open water - freshwater,NaN,7,3
197,137387434.0,197,33.947728,-9.568113,0.0,1.0,2018,2-12,0,1,0,Open water - freshwater,NaN,1,2
198,137387435.0,198,34.424376,-9.843084,0.0,1.0,2018,0,"1,2,7,8-11","3,4,5,6,12",0,Forest/woodlands,NaN,12,2


In [36]:
#Read the ground truth data following analysis step 
WOfSSample = '../Supplementary_data/Validation/Refined/AFRIGIST/Groundtruth_AfriGIST_PointBased_5D.csv'
df2= pd.read_csv(WOfSSample,delimiter=",")

In [37]:
SampleCount= df2.groupby('PLOT_ID',as_index=False,sort=False).last()
SampleCount 

,PLOT_ID,Unnamed: 0,LON,LAT,FLAGGED,ANALYSES,SENTINEL2Y,STARTDATE,ENDDATE,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG,geometry,CLASS_WET,CLEAR_OBS
0,137483175.0,12,30.463813,-26.653807,0.0,1.0,2018,NaN,NaN,1-12,0,2,0,Open water - freshwater,NaN,12,1,POINT (2939339.999593767 -3281940.000625274),1.0,1.0
1,137483176.0,24,30.026031,-26.673227,0.0,1.0,2018,NaN,NaN,1-12,0,0,0,Open water - Constructed (e.g. aquaculture),NaN,12,1,POINT (2897100.000399006 -3284159.999907182),0.0,2.0
2,137483177.0,36,31.700362,-26.746737,0.0,1.0,2018,NaN,NaN,1-12,0,"1,2",0,Open water - freshwater,NaN,10,1,POINT (3058650.000408517 -3292560.000199907),1.0,2.0
3,137483178.0,50,31.937287,-26.801901,0.0,1.0,2018,NaN,NaN,1-12,0,"2,7,12",0,Open water - freshwater,NaN,10,1,POINT (3081509.999813744 -3298860.000586511),0.0,2.0
4,137483179.0,65,27.339949,-26.863925,0.0,1.0,2018,NaN,NaN,1-12,0,0,0,Open water - freshwater,NaN,12,1,POINT (2637929.999591611 -3305939.999803136),1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,137482800.0,11166,13.848290,-8.953839,0.0,1.0,2018,1/08/2018,5/08/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,9,0,POINT (1336169.999584316 -1137749.999945619),1.0,1.0
595,137482801.0,11189,13.023406,-9.025955,0.0,1.0,2018,1/09/2018,5/09/2018,"5,11","1-4,6-10,12","1-4,6-10,12",0,Wetlands - marine (e.g. mangroves),na,9,0,POINT (1256580.000172345 -1146840.00002571),1.0,2.0
596,137482802.0,11211,13.607012,-9.155465,0.0,1.0,2018,1/09/2018,5/09/2018,8,"1-7,9-12","1-7,9-12",0,Open water - freshwater,na,9,0,POINT (1312890.00022547 -1163160.00004016),1.0,1.0
597,137482803.0,11240,20.258942,-9.360062,0.0,1.0,2018,1/09/2018,5/09/2018,1-12,0,"1-4,6-8,10-12",0,Open water - freshwater,NaN,12,1,POINT (1954709.999782347 -1188930.000040025),1.0,2.0


In [38]:
#Read the ground truth data following buffering inland from the coast 
Inland= '../Supplementary_data/Validation/Refined/AFRIGIST/AfriGIST_inland_5D.csv'
df3 = pd.read_csv(Inland,delimiter=",")

In [39]:
InlandCount= df3.groupby('PLOT_ID',as_index=False,sort=False).last()
InlandCount 

,PLOT_ID,Unnamed: 0,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION,geometry
0,137483175.0,6,30.463813,-26.653807,Open water - freshwater,12,1,1.0,1.0,1,POINT (30.46381301 -26.65380659)
1,137483176.0,14,30.026031,-26.673227,Open water - Constructed (e.g. aquaculture),12,1,0.0,2.0,0,POINT (30.02603057 -26.67322664)
2,137483177.0,19,31.700362,-26.746737,Open water - freshwater,10,1,1.0,2.0,1,POINT (31.70036188 -26.74673726)
3,137483178.0,24,31.937287,-26.801901,Open water - freshwater,10,1,0.0,2.0,0,POINT (31.93728675 -26.80190076)
4,137483179.0,32,27.339949,-26.863925,Open water - freshwater,12,1,1.0,3.0,1,POINT (27.33994919 -26.86392537)
...,...,...,...,...,...,...,...,...,...,...,...
532,137482799.0,2758,13.254423,-8.829394,Urban (Settlements/ roads),9,0,0.0,1.0,0,POINT (13.25442329 -8.82939446)
533,137482800.0,2762,13.848290,-8.953839,Open water - freshwater,9,0,1.0,1.0,1,POINT (13.8482901 -8.953838786)
534,137482802.0,2766,13.607012,-9.155465,Open water - freshwater,9,0,1.0,1.0,1,POINT (13.60701228 -9.155465)
535,137482803.0,2772,20.258942,-9.360062,Open water - freshwater,12,1,1.0,2.0,1,POINT (20.25894246 -9.360061605)


In [2]:
print(datacube.__version__)

1.8.2.dev7+gdcab0e02


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)